In [2]:
import torch
from torchvision import transforms
from PIL import Image
import os
import json
import cv2
from ultralytics import YOLO
import numpy as np
import csv
import shutil

In [3]:
# Load pre-trained Grounding DINO model(Swin-B)

# Load pre-trained Grounding DINO model
HOME = "/home/lsh/zero-shot_object_detection-24.03.18"

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinB_cfg.py")


WEIGHTS_NAME = "groundingdino_swinb_cogcoor.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)

%cd {HOME}/GroundingDINO

from groundingdino.util.inference import load_model, load_image, predict, annotate

model_GroundingDINO = load_model(CONFIG_PATH, WEIGHTS_PATH)

/home/lsh/zero-shot_object_detection-24.03.18/GroundingDINO


/home/lsh/anaconda3/envs/lsh_plate/lib/python3.7/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
class Label_Generator:
    def __init__(self,input_txt_dir,input_img_dir,output_dir):
        self.input_txt_dir = input_txt_dir
        self.input_img_dir = input_img_dir
        self.output_dir = output_dir
        self.csv_license_plate_only_dir = 'labels/license_plate_only/val'
        self.img_dir = 'images/val'
        self.csv_vehicle_n_plate_dir = 'labels/vehicle_n_plate/val'

        # 경로 적절하게 잘 합쳐주는 과정이 필요함
        os.makedirs(os.path.join(self.output_dir, self.csv_license_plate_only_dir), exist_ok=True)
        os.makedirs(os.path.join(self.output_dir, self.img_dir), exist_ok=True)
        os.makedirs(os.path.join(self.output_dir, self.csv_vehicle_n_plate_dir), exist_ok=True)
        
    
    def read_vehicle_box(self, image_path):
        #받은 img path를 file path로 바꿔주는 과정
        image_name = os.path.basename(image_path).split(".")[0]
        filepath = os.path.join(self.csv_vehicle_n_plate_dir, f"{image_name}.txt")
        
        vehicle_boxes = []
        with open(filepath, 'r') as file:
            lines = file.readlines()
            for line in lines:
                box_info = line.strip().split()
                x,y,w,h = map(float, box_info[1:])
                clist = [x,y,w,h]
                vehicle_boxes.append(clist)
        return vehicle_boxes
    
    def crop_image(self, image_path, x,y,w,h):
        image = cv2.imread(image_path)
        image_height, image_width, _ = image.shape
        x_min = int((x - w / 2) * image_width)
        y_min = int((y - h / 2) * image_height)
        x_max = int((x + w / 2) * image_width)
        y_max = int((y + h / 2) * image_height)
        cropped_image = image[y_min:y_max, x_min:x_max]
        return cropped_image

    def generate_license_plate_label(self, image_path):
        TEXT_PROMPT = "license plate of vehicle"
        BOX_TRESHOLD = 0.35 # 실험적으로 도출한 값
        TEXT_TRESHOLD = 0.25

        # 그 이미지에 해당하는 csv파일을 불러와 boxlist를 저장한다
        vehicle_boxes = self.read_vehicle_box(image_path)
        
        # for문 : box 리스트 하나에 대해 실행. 이미지를 그것에 맞게 crop 한 후 GD 진행.
        for vehicle_box in vehicle_boxes:
            cropped_image = self.crop_image(image_path, *vehicle_box)
            #image_source, image = load_image(image_path)
            # 이 부분이 명확하게 되는지 확인 필요
            
            
            boxes, logits, phrases = predict(
                model=model_GroundingDINO,
                image=cropped_image,
                caption=TEXT_PROMPT,
                box_threshold=BOX_TRESHOLD,
                text_threshold=TEXT_TRESHOLD
            )
            
            # 이 완성된 boxes는 crop한 image에서의 상대 좌표이다. 원래의 image에 대한
            #상대 좌표로 바꿔줄 필요가 있다
            
            # 완성된 결과물에 대한 작업 위해서
            image_name = os.path.basename(image_path).split(".")[0]
            self.convert_class_name(image_name)
            if phrases != []:
                self.save_license_plate_only(boxes, phrases, image_name)
                self.save_vehicle_n_plate(boxes, phrases, image_name)  
                self.save_image(image_name)
        
        # 그리고 모든 box에 대해 GD를 다 진행하면서 얻어낸 결과물들을 리스트 형태로 합침.
        # 이때 원래 이미지의 상대적 위치값으로 변화해줄 필요가 있음.
        
        
        # 만약 합쳐진 리스트가 빈 리스트가 아니라면, 그때 추가함



        image_source, image = load_image(image_path)

        boxes, logits, phrases = predict(
            model=model_GroundingDINO,
            image=cropped_image,
            caption=TEXT_PROMPT,
            box_threshold=BOX_TRESHOLD,
            text_threshold=TEXT_TRESHOLD
        )
        image_name = os.path.basename(image_path).split(".")[0]
        self.convert_class_name(image_name)
        if phrases != []:
            self.save_license_plate_only(boxes, phrases, image_name)
            self.save_vehicle_n_plate(boxes, phrases, image_name)
            self.save_image(image_name)
        
        
        
    def save_license_plate_only(self, boxes, phrases, image_name):  # 완성
        DINO_labels = []
        for box, phrase in zip(boxes, phrases):
            if phrase == 'license plate':
                box_info = box
                label = box_info.float().tolist()
                label.insert(0, phrase)
                DINO_labels.append(label)   # 이 단계까지 완료 시 plate,x,y,w,h 리스트 완성

        csv_filename = os.path.join(self.output_dir, self.csv_license_plate_only_dir, f"{image_name}.csv")
        with open(csv_filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            for DINO_label in DINO_labels:
                writer.writerow(DINO_label)
    
    
    
    def save_image(self, image_name):
        input_image_path = os.path.join(self.input_img_dir, f"{image_name}.jpg")
        output_image_path = os.path.join(self.output_dir ,self.img_dir, f"{image_name}.jpg")
        print(self.input_img_dir)
        print(output_image_path)
        shutil.copy(input_image_path, output_image_path)
    
    
    
    def convert_class_name(self,image_name):  
        # csv파일을 읽어서 맨 앞을 index에서 name으로 바꾸는 함수
        txt_file_original = os.path.join(self.input_txt_dir, f"{image_name}.txt")
        csv_file_changed = os.path.join(self.output_dir, self.csv_vehicle_n_plate_dir, f"{image_name}.csv")
        vehicle_boxes = []
        with open(txt_file_original, mode='r') as original_txt:
            lines = original_txt.readlines()
            for line in lines:
                index, *box_info = line.strip().split()
                label = []
                vehicle_box = []
                if index == '2':
                    label.append('car')
                elif index == '5':
                    label.append('bus')
                elif index == '7':
                    label.append('truck')
                else:
                    pass
                for coordinate in box_info:
                    label.append(coordinate)
                if label[0] in ['car', 'bus', 'truck']:
                    vehicle_boxes.append(label)
        if vehicle_boxes != []: # vehicle이 하나라도 담겼을 때만 csv 파일로 저장
            with open(csv_file_changed,mode ='w', newline='') as changed_csv:
                writer = csv.writer(changed_csv)
                for vehicle_box in vehicle_boxes:
                    writer.writerow(vehicle_box)
                    
                
    
    def save_vehicle_n_plate(self, boxes, phrases, image_name):
        DINO_labels = []
        if phrases != []:
            for box, phrase in zip(boxes, phrases):
                if phrase == 'license plate':
                    box_info = box
                    label = box_info.float().tolist()
                    label.insert(0, phrase)
                    DINO_labels.append(label)   # 이 단계까지 완료 시 plate,x,y,w,h 리스트 완성

        csv_filename = os.path.join(self.output_dir, self.csv_vehicle_n_plate_dir, f"{image_name}.csv")
        with open(csv_filename, mode='a', newline='') as file:
            writer = csv.writer(file)
            for DINO_label in DINO_labels:
                writer.writerow(DINO_label)
    

In [72]:
input_txt_dir ='/home/lsh/coco2017_vehicle/labels/val'
input_img_dir = '/home/lsh/coco2017_vehicle/images/val'
output_dir = '/home/lsh/Project_label_generation/output'

test_input_txt_dir = '/home/lsh/label_generation_test_dataset/labels'
test_input_img_dir = '/home/lsh/label_generation_test_dataset/images'

generator = Label_Generator(test_input_txt_dir, test_input_img_dir, output_dir)


for image_name in os.listdir(test_input_img_dir):
    image_path = os.path.join(test_input_img_dir, image_name)
    
    generator.generate_license_plate_label(image_path)

/home/lsh/label_generation_test_dataset/images
/home/lsh/Project_label_generation/output/images/val/000000001532.jpg
/home/lsh/label_generation_test_dataset/images
/home/lsh/Project_label_generation/output/images/val/000000006723.jpg
/home/lsh/label_generation_test_dataset/images
/home/lsh/Project_label_generation/output/images/val/000000006040.jpg
/home/lsh/label_generation_test_dataset/images
/home/lsh/Project_label_generation/output/images/val/000000007386.jpg
/home/lsh/label_generation_test_dataset/images
/home/lsh/Project_label_generation/output/images/val/000000001584.jpg
/home/lsh/label_generation_test_dataset/images
/home/lsh/Project_label_generation/output/images/val/000000002006.jpg
/home/lsh/label_generation_test_dataset/images
/home/lsh/Project_label_generation/output/images/val/000000002006 copy.jpg
/home/lsh/label_generation_test_dataset/images
/home/lsh/Project_label_generation/output/images/val/000000005037.jpg
/home/lsh/label_generation_test_dataset/images
/home/lsh/Pr